## **IMPORTING GOOGLE** **DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%mkdir /content/drive/MyDrive/object/

## **INSTALLING REQUIRED PACKAGES**

In [ ]:
! git clone https://github.com/tensorflow/models

In [ ]:
%cd /content/drive/MyDrive/object/

In [ ]:
!wget https://github.com/protocolbuffers/protobuf/releases/download/v3.15.3/protoc-3.15.3-linux-x86_64.zip
!mkdir protoc
!unzip protoc-3.15.3-linux-x86_64.zip -d  protoc

In [ ]:
! chmod 755 /content/drive/MyDrive/object/protoc/bin/protoc 

In [ ]:
%cd /content/drive/MyDrive/object/models/research/

In [ ]:
! /content/drive/MyDrive/object/protoc/bin/protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%cd /content/drive/MyDrive/object/
! pip install cython
! git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
%cd /content/drive/MyDrive/object/cocoapi/PythonAPI/

In [ ]:
! make

In [ ]:
! cp -r pycocotools /content/drive/MyDrive/object/models/research/

In [ ]:
%cd /content/drive/MyDrive/object/models/research/

In [ ]:
! cp object_detection/packages/tf2/setup.py .

In [ ]:
! python -m pip install .

In [ ]:
! python -m pip install .

In [ ]:
! python object_detection/builders/model_builder_tf2_test.py

In [ ]:
from pycocotools.coco import COCO
from pycocotools import mask

In [ ]:
%cd /content/drive/MyDrive/object/

# **CLONING GITHUB REPO FOR TRAINING MODEL**

In [ ]:
! git clone https://github.com/bharanivijay69/FireAndSmokeDetection

In [ ]:
cd FireAndSmokeDetection

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = 'Tensorflow/workspace/annotations'
IMAGE_PATH = 'Tensorflow/workspace/images'
MODEL_PATH = 'Tensorflow/workspace/models'
PRETRAINED_MODEL_PATH ='Tensorflow/workspace/pre-trained-models'
CONFIG_PATH = 'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = 'Tensorflow/workspace/models/my_ssd_mobnet/'

In [ ]:
labels = [{'name':'fire', 'id':1}, {'name':'smoke', 'id':2}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
labels

**COPY IMAGES TO Tensorflow/workspace/images IN GOOGLE** **DRIVE**

In [ ]:
!python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

In [ ]:
!mkdir {'Tensorflow/workspace/models/my_ssd_mobnet'}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {'Tensorflow/workspace/models/my_ssd_mobnet'}

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = 'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [ ]:
config

In [ ]:
print("""python3 {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,'my_ssd_mobnet',MODEL_PATH,'my_ssd_mobnet'))

In [ ]:
!python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000

# **TRAINING OF MODEL IS DONE**